# B92 Quantum Key Distribution Simulation

## Overview
This notebook implements a simulation of the B92 quantum key distribution protocol, which uses only two non-orthogonal quantum states for secure key distribution.

## Theory
- Alice encodes: |0⟩ for bit 0, |+⟩ for bit 1
- Bob measures with projectors orthogonal to Alice's states
- Only conclusive measurements are kept for the key
- Security relies on quantum indistinguishability

## 1. Imports and Quantum State Definitions

In [0]:
import numpy as np
from numpy import random
import matplotlib.pyplot as plt
from scipy.stats import norm, binom
import time

# Quantum states (as 2D complex vectors)
sqrt2 = np.sqrt(2)

psi0 = np.array([1.0, 0.0], dtype=complex)                # Alice state for bit 0
psi1 = np.array([1.0/sqrt2, 1.0/sqrt2], dtype=complex)    # Alice state for bit 1 (|+>)

# Vectors orthogonal to Alice's states
phi0 = np.array([1.0/sqrt2, -1.0/sqrt2], dtype=complex)   # orthogonal to psi1
phi1 = np.array([0.0, 1.0], dtype=complex)                # orthogonal to psi0

## 2. Core Quantum Functions

In [0]:
def project_probability(proj_vec, state_vec):
    """Probability that projector onto proj_vec clicks for input state state_vec."""
    amp = np.vdot(proj_vec, state_vec)
    return np.abs(amp)**2

def encode_state(bit):
    """Return the 2-vector representing Alice's state for a given bit (0 or 1)."""
    return psi0 if bit == 0 else psi1

def bob_measure_one(projector_choice, incoming_state):
    """
    Bob chooses one projector to test (phi0 or phi1).
    Returns: bit value if conclusive, None if inconclusive.
    """
    if projector_choice == 0:
        p = project_probability(phi0, incoming_state)
        if random.random() < p:
            return 0
        else:
            return None
    else:
        p = project_probability(phi1, incoming_state)
        if random.random() < p:
            return 1
        else:
            return None

## 3. B92 Protocol Simulation

In [0]:
def random_b92(length):
    """Prepare random Alice bits and Bob's measurement choices."""
    alice_bits = random.randint(2, size=length)
    bob_choice = random.randint(2, size=length)
    return alice_bits, bob_choice

def simulate_b92_no_eve(alice_bits, bob_choice):
    """Simulate B92 protocol without eavesdropper."""
    bob_results = []
    for i in range(len(alice_bits)):
        state = encode_state(alice_bits[i])
        result = bob_measure_one(bob_choice[i], state)
        bob_results.append(result)
    return bob_results

def simulate_b92_with_eve(alice_bits, bob_choice):
    """Simulate B92 protocol with Eve intercept-resend attack."""
    n = len(alice_bits)
    eve_choices = random.randint(2, size=n)
    eve_results = []
    bob_results = []
    
    for i in range(n):
        incoming = encode_state(alice_bits[i])
        eve_result = bob_measure_one(eve_choices[i], incoming)
        eve_results.append(eve_result)
        
        if eve_result == 0:
            resend_state = psi0
        elif eve_result == 1:
            resend_state = psi1
        else:
            resend_state = psi0 if random.randint(2) == 0 else psi1

        bob_res = bob_measure_one(bob_choice[i], resend_state)
        bob_results.append(bob_res)

    return bob_results, eve_results

## 4. Key Generation and Analysis

In [0]:
def create_key_b92(alice_bits, bob_results):
    """Create final key from conclusive measurement results."""
    res_key = []
    indices = []
    count_true = 0
    count_false = 0
    
    for i, br in enumerate(bob_results):
        if br is not None:
            res_key.append(br)
            indices.append(i)
            if br == alice_bits[i]:
                count_true += 1
            else:
                count_false += 1
                
    total = count_true + count_false
    accuracy = count_true / len(alice_bits)
    
    return {
        'res_key': res_key,
        'indices': indices,
        'len_key': len(res_key),
        'total_conclusive': total,
        'count_true': count_true,
        'count_false': count_false,
        'accuracy': accuracy
    }

def print_stats_b92(alice_bits, bob_results):
    """Print detailed statistics of the B92 protocol execution."""
    info = create_key_b92(alice_bits, bob_results)
    n = len(alice_bits)
    total_conc = info['total_conclusive']
    count_false = info['count_false']
    count_true = info['count_true']
    QBER = count_false / total_conc if total_conc > 0 else 0
    
    print(f'Accuracy: {info["accuracy"]:.6f}')
    print('Total bits sent by Alice:', n)
    print('Conclusive (kept) outcomes:', total_conc)
    print('Matching bits (conclusive & correct):', count_true)
    print('Mismatched bits (conclusive & wrong):', count_false)
    print(f'QBER on the conclusive subset: {QBER:.6f}')
    print('---')

## 5. Message Encryption Functions

In [0]:
def string_to_binary(string):
    """Convert string to list of binary bits."""
    binary_list = []
    for char in string:
        bin_char = format(ord(char), '08b')
        for bit in bin_char:
            binary_list.append(int(bit))
    return binary_list

def binary_to_string(binary_list):
    """Convert list of binary bits back to string."""
    chars = []
    for i in range(0, len(binary_list), 8):
        chunk = binary_list[i:i+8]
        if len(chunk) < 8:
            chunk = chunk + [0]*(8-len(chunk))
        char_int = int(''.join(map(str, chunk)), 2)
        chars.append(chr(char_int))
    return ''.join(chars)

def encryption(data, key):
    """Encrypt data using XOR with key."""
    if len(data) != len(key):
        key = key[:len(data)]
    return [data[i] ^ key[i] for i in range(len(data))]

def decryption(message, key):
    """Decrypt message using XOR with key."""
    return encryption(message, key)

## 6. Monte Carlo QBER Analysis

In [0]:
def compute_qber_b92(alice_bits, bob_results):
    """Compute QBER only on conclusive events."""
    indices = [i for i in range(len(bob_results)) if bob_results[i] is not None]
    if len(indices) == 0:
        return 0.0
    errors = sum(alice_bits[i] != bob_results[i] for i in indices)
    return errors / len(indices)

def monte_carlo_b92_with_eve(num_trials=500, key_length=5000, bins=30):
    """Run Monte Carlo simulation to analyze QBER distribution with Eve."""
    qbers = []
    detections = 0
    
    print("\nRunning Monte Carlo B92 simulation with Eve...")
    start_time = time.time()

    for _ in range(num_trials):
        alice_bits, bob_choice = random_b92(key_length)
        bob_bits, eve_bits = simulate_b92_with_eve(alice_bits, bob_choice)
        qber = compute_qber_b92(alice_bits, bob_bits)
        qbers.append(qber)
        
        if qber >= 0.05:
            detections += 1

    end_time = time.time()
    runtime = end_time - start_time
    mean_qber = np.mean(qbers)
    std_qber = np.std(qbers)
    detection_probability = detections / num_trials

    print("\n==============================")
    print("Monte Carlo B92 with Eve – QBER Summary")
    print(f"Trials: {num_trials}")
    print(f"Bits per trial: {key_length}")
    print(f"Mean QBER (conclusive subset): {mean_qber:.4f}")
    print(f"Standard Deviation: {std_qber:.4f}")
    print(f"Eve detected in {detections}/{num_trials} trials")
    print(f"Detection Probability: {detection_probability:.3f}")
    print(f"Runtime: {runtime:.2f} seconds")
    print("==============================")

    # Plot histogram with distribution fits
    plt.figure(figsize=(10, 6))
    plt.hist(qbers, bins=bins, density=True, alpha=0.6, label='QBER Histogram', color='purple')

    # Binomial and Gaussian fits
    avg_conclusive = np.mean([sum(1 for r in simulate_b92_no_eve(*random_b92(key_length)) if r is not None) for _ in range(20)])
    N = max(1, int(avg_conclusive))
    p = mean_qber
    
    k_vals = np.arange(0, N+1)
    q_vals = k_vals / N
    pmf_vals = binom.pmf(k_vals, N, p)
    if len(q_vals) > 1:
        pdf_scaled = pmf_vals / (q_vals[1] - q_vals[0])
        plt.plot(q_vals, pdf_scaled, 'r--', linewidth=2, label='Binomial Fit (approx)')

    x = np.linspace(0, 1, 200)
    plt.plot(x, norm.pdf(x, mean_qber, std_qber), 'g-.', label='Gaussian Fit')

    plt.xlabel('QBER (on conclusive bits)')
    plt.ylabel('Probability Density')
    plt.title('B92 QBER Distribution with Eve (Intercept-Resend Attack)')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()

    return {
        'qbers': qbers,
        'mean_qber': mean_qber,
        'std_qber': std_qber,
        'detection_probability': detection_probability,
        'runtime': runtime
    }

## 7. Main Simulation

In [0]:
def run_b92_simulation():
    """Run the complete B92 simulation with user input."""
    
    while True:
        try:
            length = int(input("Enter the number of bits to send: "))
            if length <= 0:
                raise ValueError
            break
        except ValueError:
            print("Please enter a valid positive integer.")

    eve_input = input("Include Eve in the simulation? (yes/no): ").strip().lower()
    include_eve = eve_input in ["yes", "y"]

    message = input("Enter the message Alice wants to send: ")
    binary_message = string_to_binary(message)

    alice_bits, bob_choice = random_b92(length)

    if not include_eve:
        bob_results = simulate_b92_no_eve(alice_bits, bob_choice)
        print("\n--- B92 Simulation without Eve ---")
        print_stats_b92(alice_bits, bob_results)
        key_info = create_key_b92(alice_bits, bob_results)
        key = key_info['res_key']
        print("Key length (conclusive bits):", len(key))
    else:
        bob_results, eve_results = simulate_b92_with_eve(alice_bits, bob_choice)
        print("\n--- B92 Simulation with Eve (intercept-resend) ---")
        print_stats_b92(alice_bits, bob_results)
        key_info = create_key_b92(alice_bits, bob_results)
        key = key_info['res_key']
        print("Key length (conclusive bits):", len(key))

        run_mc = input("\nRun Monte Carlo QBER analysis with Eve? (yes/no): ").strip().lower()
        if run_mc in ["yes", "y"]:
            try:
                num_trials = int(input("Enter number of Monte Carlo trials (e.g. 500): "))
            except ValueError:
                num_trials = 500
            monte_carlo_b92_with_eve(num_trials=num_trials, key_length=length)

    # Encrypt and decrypt message
    if len(key) < len(binary_message):
        print(f"\nWarning: Key length ({len(key)}) is shorter than message length ({len(binary_message)})")
    
    key_for_message = key[:len(binary_message)]
    encrypted_message = encryption(binary_message, key_for_message)
    decrypted_message = decryption(encrypted_message, key_for_message)
    decrypted_string = binary_to_string(decrypted_message)

    print("\n--- Message Transmission ---")
    print("Original message:", message)
    print("Decrypted message:", decrypted_string)
    print("Encryption successful:", message == decrypted_string)

# Run the simulation
if __name__ == "__main__":
    run_b92_simulation()